In [2]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score)

sys.path.append("../code/")
from models.layers import *

In [3]:
folder_data = "../res/data/"
folder_nano = os.path.join(folder_data, "nano")
folder_nano_numpy = os.path.join(folder_data, "nano_volumes")
filename_train_labels = os.path.join(folder_data, "train_labels.csv")
filename_train_metadata = os.path.join(folder_data, "train_metadata.csv")
filename_test_metadata = os.path.join(folder_data, "test_metadata.csv")
folder_test_numpy = "/media/igorkozlovskii/Elements/Projects/stall_catchers/test_numpy/"


def get_simple_model(input_shape=(32, 32, 32)):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Reshape(input_shape + (1,)),
        ConvBlock_1(32),
        ConvBlock_1(64),
        ConvBlock_2(64),
        ConvBlock_2(128),
        Conv3D_bn(256, kernel_size=2, padding="valid"),
        tf.keras.layers.Reshape((256,)),
        Dense(1, activation=tf.keras.activations.sigmoid),
    ])
    return model

def get_binary(y, threshold=0.5):
    return y >= threshold
    
def get_scores(y_true, y_pred, threshold=0.5):
    y_pred_bin = get_binary(y_pred, threshold=threshold)
    scores = {
        "accuracy": accuracy_score(y_true, y_pred_bin),
        "precision": precision_score(y_true, y_pred_bin),
        "recall": recall_score(y_true, y_pred_bin),
        "f1": f1_score(y_true, y_pred_bin),
        "auc": roc_auc_score(y_true, y_pred),
        "mcc": matthews_corrcoef(y_true, y_pred_bin),
    }
    return scores

def print_scores(*args, **kwargs):
    scores = get_scores(*args, **kwargs)
    for name, value in scores.items():
        print("{:10s} {:.3f}".format(name, value))
    return scores

In [4]:

names = [f.name for f in os.scandir(folder_nano)]
data_train_labels = pd.read_csv(filename_train_labels)
data_train_labels = data_train_labels[
    (data_train_labels.filename.isin(names))]
names = data_train_labels.filename.values.tolist()
labels = data_train_labels.stalled.values

y = labels.copy()
X = []
for name in tqdm(names):
    filename = os.path.join(folder_nano_numpy, name.split(".")[0] + ".npy")
    X.append(np.load(filename))
X = np.array(X, np.float32)

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=0)

print(X_train.shape, y_train.shape, y_train.sum(), y_train.sum() / len(y_train))
print(X_test.shape, y_test.shape, y_test.sum(), y_test.sum() / len(y_test))


(1130, 32, 32, 32) (1130,) 565 0.5
(283, 32, 32, 32) (283,) 141 0.49823321554770317


In [6]:
model = get_simple_model(X.shape[1:4])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.5,
                              patience=5, min_lr=1e-6,),
    tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=10)
]
model.fit(X_train, y_train, 
    batch_size=16, 
    epochs=10, 
    validation_data=(X_test, y_test),
    callbacks=callbacks,)

Train on 1130 samples, validate on 283 samples
Epoch 1/10
1130/1130 [==============================] - 10s 9ms/sample - loss: 0.7397 - acc: 0.5841 - val_loss: 0.9941 - val_acc: 0.5018
Epoch 2/10
1130/1130 [==============================] - 9s 8ms/sample - loss: 0.6582 - acc: 0.6398 - val_loss: 0.7263 - val_acc: 0.5018
Epoch 3/10
1130/1130 [==============================] - 9s 8ms/sample - loss: 0.6134 - acc: 0.6858 - val_loss: 0.7312 - val_acc: 0.5053
Epoch 4/10
1130/1130 [==============================] - 9s 8ms/sample - loss: 0.5664 - acc: 0.7195 - val_loss: 0.7182 - val_acc: 0.5088
Epoch 5/10
1130/1130 [==============================] - 9s 8ms/sample - loss: 0.5573 - acc: 0.7336 - val_loss: 0.7337 - val_acc: 0.5442
Epoch 6/10
1130/1130 [==============================] - 9s 8ms/sample - loss: 0.5267 - acc: 0.7416 - val_loss: 0.6593 - val_acc: 0.6466
Epoch 7/10
1130/1130 [==============================] - 9s 8ms/sample - loss: 0.4696 - acc: 0.7841 - val_loss: 0.9779 - val_acc: 0.6113


In [7]:
y_train_preds = model.predict(X_train)
y_test_preds = model.predict(X_test)

print(y_train_preds.shape)
print(y_test_preds.shape)

scores_train = print_scores(y_train, y_train_preds)
print()
scores_test = print_scores(y_test, y_test_preds)

(1130, 1)
(283, 1)
accuracy   0.807
precision  0.981
recall     0.627
f1         0.765
auc        0.908
mcc        0.659

accuracy   0.664
precision  0.780
recall     0.454
f1         0.574
auc        0.746
mcc        0.361


In [44]:

data_test_metadata = pd.read_csv(filename_test_metadata)
names_test = data_test_metadata.filename.values.tolist()
y_test_pred_subm = []
for name in tqdm(names_test):
    filename = os.path.join(folder_test_numpy, name.split(".")[0] + ".npy")
    vol = np.load(filename)
    vol = np.array([vol])
    y_test_pred_subm.append(model.predict(vol)[0, 0])
    
y_test_pred_subm = np.array(y_test_pred_subm)

In [43]:
model.predict(vol)[0, 0]

0.0007028469

In [55]:
y_subm = np.array(y_test_pred_subm >= 0.999, np.int32)
print(np.mean(y_test_pred_subm))
print(np.mean(y_subm), np.sum(y_subm), np.sum(y_subm) / len(y_subm))
data_subm = pd.DataFrame({"filename": names_test, "stalled": y_subm})
data_subm.to_csv(os.path.join("submission_1.csv"), index=False)

0.35464847
0.06574858757062146 931 0.06574858757062146


In [56]:
data_subm.head()

,filename,stalled
0,100032.mp4,0
1,100037.mp4,0
2,100139.mp4,1
3,100182.mp4,0
4,100214.mp4,0
